In [1]:
import numpy as np
from tkinter import *
from tkinter import colorchooser

In [3]:
def makeDefaultWindow(title, geometry):
    window = Tk()
    window.geometry(geometry)
    window.title(title)
    window.resizable(width=False, height=False)
    return window

In [53]:
def create_label(window, default_text, font_size, bg='default'):
    if bg == 'default':
        bg = window.cget('bg')
        
    label = Label(window, text=default_text, bg=bg)
    label.config(font=("Courier", font_size))
    return label

def create_textbox(window, height, width, default_text, font_size):
    textbox = Text(window, height=height, width=width)
    textbox.insert(1.0, default_text)
    textbox.configure(font=("Courier", font_size))
    return textbox

In [130]:
def get_textbox_to_num(textbox):
    txt = textbox.get('1.0', 'end').replace('\n', '')
    return int(txt)

def get_textbox(textbox):
    txt = textbox.get('1.0', 'end').replace('\n', '')
    return txt

def modify_textbox(textbox, txt):
    textbox.delete('1.0', 'end')
    textbox.insert(1.0, txt)

In [276]:
class GridEnv():
    def __init__(self, title='Application', cnt_grid_x = 10, cnt_grid_y = 10, grid_x = 400, grid_y = 400,
                max_grid = 30, min_grid = 2, marginX = 5, marginY = 5):
        self.window = makeDefaultWindow(title=title, geometry='800x800+500+100')
        
        self.cnt_grid_x = cnt_grid_x
        self.cnt_grid_y = cnt_grid_y
        self.grid_x = 400
        self.grid_y = 400
        self.recWidth = self.grid_x / self.cnt_grid_x
        self.recHeight = self.grid_y / self.cnt_grid_y
        
        self.max_grid = max_grid
        self.min_grid = min_grid
        self.marginX = marginX
        self.marginY = marginY
        
        self.grid_canvas = None
        self.create_canvas = None
        
        self.furniture_list = {}
        self.furniture_objlist = []
        self.grids = []
        
        self.create_resize_grid_layout()
        self.make_grid()
        self.make_create_layout()
        
    def runApp(self):
        self.window.mainloop()
        
    def create_resize_grid_layout(self):
        label = create_label(window=self.window, default_text='RoomSize', font_size=14)
        label.place(x=100, y=50)
        
        label = create_label(window=self.window, default_text='X', font_size=16)
        label.place(x=150, y=100)
        
        self.grid_x_textbox = create_textbox(window=self.window, height=1, width=7, default_text='10', font_size=16)
        self.grid_x_textbox.place(x=50, y=100)
        
        self.grid_y_textbox = create_textbox(window=self.window, height=1, width=7, default_text='10', font_size=16)
        self.grid_y_textbox.place(x=170, y=100)
        
        submit_bt = Button(self.window, text='Submit', command = self.submit)
        submit_bt.place(x=280, y=100)
        
    def submit(self):
        for f in self.furniture_objlist:
            f.destroy()
        
        grid_x = get_textbox_to_num(self.grid_x_textbox)
        grid_y = get_textbox_to_num(self.grid_y_textbox)
        
        if grid_x > self.max_grid:
            grid_x = self.max_grid
            modify_textbox(self.grid_x_textbox, str(self.max_grid))
        elif grid_x < self.min_grid:
            grid_x = self.min_grid
            modify_textbox(self.grid_x_textbox, str(self.min_grid))
            
        if grid_y > self.max_grid:
            grid_y = self.max_grid
            modify_textbox(self.grid_y_textbox, str(self.max_grid))
        elif grid_y < self.min_grid:
            grid_y = self.min_grid
            modify_textbox(self.grid_y_textbox, str(self.min_grid))
            
        self.modify_size(grid_x = grid_x, grid_y = grid_y)
        self.make_grid()
        
    def modify_size(self, grid_x, grid_y):
        self.cnt_grid_x = grid_x
        self.cnt_grid_y = grid_y
        
        self.recWidth = self.grid_x / self.cnt_grid_x
        self.recHeight = self.grid_y / self.cnt_grid_y
        
    def make_grid(self):
        if self.grid_canvas != None:
            self.grid_canvas.destroy()
            
        self.grid_canvas = Canvas(self.window, width=self.grid_x + 50, height=self.grid_y + 5)
        self.grid_canvas.place(x=20, y=200)
        
        for i in range(self.cnt_grid_x):
            for j in range(self.cnt_grid_y):
                rect_id = self.grid_canvas.create_rectangle(i*self.recWidth+self.marginX, j*self.recHeight+self.marginY,
                                                      (i+1)*self.recWidth+self.marginX, (j+1)*self.recHeight+self.marginY,
                                                      fill='white', tags='grid')
                
    def choose_color(self, color_rect):
        self.color_code = colorchooser.askcolor(title='Choose Color')
        self.create_canvas.itemconfig(self.create_canvas.find_withtag(color_rect), fill=self.color_code[1])
        
    def make_create_layout(self):
        self.create_canvas = Canvas(self.window, width=250, height=600, bg='cyan', highlightthickness=0)
        self.create_canvas.place(x=500, y=50)
        
        label = create_label(window=self.create_canvas, default_text='Name', font_size=14)
        label.place(x=10, y=50)
        name_textbox = create_textbox(window=self.create_canvas, height=1, width=10, default_text='', font_size=16)
        name_textbox.place(x=80, y=50)
        
        label = create_label(window=self.create_canvas, default_text='Color', font_size=14)
        label.place(x=10, y=100)
        self.color_code = ((255, 255, 255), '#ffffff')
        color_rect = self.create_canvas.create_rectangle(200, 100, 230, 130, fill=self.color_code[1])
        color_button = Button(self.create_canvas, text='Select', height=1, width=13, 
                              command=lambda:self.choose_color(color_rect))
        color_button.place(x=80, y=100)
        
        label = create_label(window=self.create_canvas, default_text='Size', font_size=14)
        label.place(x=10, y=150)
        label = create_label(window=self.create_canvas, default_text='X', font_size=14)
        label.place(x=120, y=150)
        label = create_label(window=self.create_canvas, default_text='X', font_size=14)
        label.place(x=180, y=150)
        size_x_textbox = create_textbox(window=self.create_canvas, height=1, width=3, default_text='1', font_size=13)
        size_y_textbox = create_textbox(window=self.create_canvas, height=1, width=3, default_text='1', font_size=13)
        size_z_textbox = create_textbox(window=self.create_canvas, height=1, width=3, default_text='1', font_size=13)
        size_x_textbox.place(x=80, y=150)
        size_y_textbox.place(x=140, y=150)
        size_z_textbox.place(x=200, y=150)
        
        create_btn = Button(self.create_canvas, text='Create', width=30, height=2, bg='cyan', 
                            command=lambda: self.create_furniture(name_textbox, size_x_textbox, size_y_textbox, size_z_textbox))
        create_btn.place(x=15, y=0)
        
    def create_furniture(self, nameT, size_xT, size_yT, size_zT):
        name = get_textbox(nameT)
        color = self.color_code[1]
        size_x = get_textbox_to_num(size_xT)
        size_y = get_textbox_to_num(size_yT)
        size_z = get_textbox_to_num(size_zT)
        
        self.add_dict(name=name, color=color, size_list=(size_x, size_y, size_z))
        
        xlist = [x+25 for x in range(0, 400, int(self.recWidth))]
        ylist = [y+205 for y in range(0, 400, int(self.recHeight))]
        
        furniture = Canvas(self.window, bg=color, width=(size_x*self.recWidth)-1, height=(size_y*self.recHeight)-1,
                          highlightthickness=0)
        furniture.place(x=0, y=0)
        furniture.bind('<B1-Motion>', self.move)
        furniture.bind('<ButtonRelease-1>', lambda event, xlist=xlist, ylist=ylist, size_x=size_x, size_y=size_y:
                       self.release(event,xlist, ylist, size_x, size_y))
        
        self.furniture_objlist.append(furniture)
        
    def add_dict(self, name, color, size_list):
        key = 'furniture ' + str(len(self.furniture_list)+1)
        self.furniture_list[key] = {}
        
        size = str(size_list[0]) + 'x' + str(size_list[1]) + 'x' + str(size_list[2])
        
        self.furniture_list[key]['name'] = name
        self.furniture_list[key]['color'] = color
        self.furniture_list[key]['size'] = size
        
    def move(self, event):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        event.widget.place(x=int(mouse_x-self.recWidth/2), y=int(mouse_y-self.recHeight/2))
    
    def release(self, event, xlist, ylist, size_x, size_y):
        mouse_x = self.window.winfo_pointerx() - self.window.winfo_rootx()
        mouse_y = self.window.winfo_pointery() - self.window.winfo_rooty()
        
        if mouse_x > 425 or mouse_y > 605 or mouse_x < 25 or mouse_y < 205:
            event.widget.place(x=mouse_x-self.recWidth/2, y=mouse_y-self.recHeight/2)
        
        else:
            for x in xlist:
                if mouse_x > x:
                    place_x = x
                    
            for y in ylist:
                if mouse_y > y:
                    place_y = y
             
            for x in reversed(xlist):
                if place_x + self.recWidth * size_x > 425:
                    place_x = x
                    
            for y in reversed(ylist):
                if place_y + self.recHeight * size_y > 605:
                    place_y = y

            event.widget.place(x=place_x+1, y=place_y+1)

In [277]:
app = GridEnv()
app.runApp()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vfgtr554\anaconda3\envs\learning\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\vfgtr554\AppData\Local\Temp/ipykernel_12376/736866676.py", line 147, in <lambda>
    self.release(event,xlist, ylist, size_x, size_y))
  File "C:\Users\vfgtr554\AppData\Local\Temp/ipykernel_12376/736866676.py", line 184, in release
    if place_y + self.recHeight * size_y > 605:
UnboundLocalError: local variable 'place_y' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\vfgtr554\anaconda3\envs\learning\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\vfgtr554\AppData\Local\Temp/ipykernel_12376/736866676.py", line 147, in <lambda>
    self.release(event,xlist, ylist, size_x, size_y))
  File "C:\Users\vfgtr554\AppData\Local\Temp/ipykernel_12376/736866676.py", line 184, in relea

In [ ]:
(25,205) to (425, 605)